In [17]:
!pip install -q openai faiss-cpu numpy

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.5 MB/s eta 0:00:00


In [30]:
from google.colab import files

uploaded = files.upload()  # select the original material file

Saving NovaTech_Product_Intro_Bilingual.txt to NovaTech_Product_Intro_Bilingual.txt


In [41]:
filename = "NovaTech_Product_Intro_Bilingual.txt"  # your file name

with open(filename, "r", encoding="utf-8") as f:
    full_text = f.read()

# using a paragraph to chunk
text_chunks = [chunk.strip() for chunk in full_text.split("\n") if chunk.strip()]
print(f"split to {len(text_chunks)} chunks")

split to 56 chunks


In [33]:
from google.colab import userdata
from openai import OpenAI
import os

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# embedding the texts
def embed_texts(texts):
    response = client.embeddings.create(
        input=texts,
        model="text-embedding-3-small"
    )
    return [item.embedding for item in response.data]

In [42]:
import faiss
import numpy as np

# generate embedding texts
embeddings = embed_texts(text_chunks)
embedding_dim = len(embeddings[0])

# create FAISS index
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings).astype("float32"))

# store FAISS index
faiss.write_index(index, "docs_index.faiss")

# store chunks
with open("docs_chunks.txt", "w", encoding="utf-8") as f:
    for line in text_chunks:
        f.write(line + "\n")

print("✅ index and chunks stored")

✅ index and chunks stored


In [35]:
import faiss
import numpy as np

# read the index and the original paragraphs
index = faiss.read_index("docs_index.faiss")

with open("docs_chunks.txt", "r", encoding="utf-8") as f:
    chunks = [line.strip() for line in f if line.strip()]

In [48]:
# judge to use AI or RAG to answer, the lower the score, the high the similarity,
# if the score<threshhold, should answer from the doc
def answer_query(query, threshold=0.9):
    # embed the query
    query_vector = np.array([embed_texts([query])[0]]).astype("float32")

    # query the FAISS index
    distances, indices = index.search(query_vector, k=1)
    top_score = distances[0][0]
    top_match = chunks[indices[0][0]]

    print(f"Top similarity distance: {top_score:.4f}")
    print(f"Top matched paragraph:\n{top_match}\n")

    # if the similarity>threshold，answer with RAG
    if top_score < threshold:
        messages = [
            {"role": "system", "content": "你是一個聰明的 AI 助理，請根據提供的上下文或你的知識回答問題。"},
            {"role": "user", "content": f"請根據以下資料回答問題：\n\n{top_match}\n\n問題：{query}"}
        ]
        mode = "📄 answer with RAG"
    else:
        # fallback：if the similarity<threshold, answer with GPT
        fallback_message = (
            "我沒有在上傳的文檔中找到與你的問題足夠相似的段落，"
            "所以改用 GPT 模式嘗試幫你回答。你也可以試著重新描述問題，"
            "或確認上傳的文檔是否包含你要問的主題。"
        )
        print(f"⚠️ {fallback_message}")
        messages = [
            {"role": "system", "content": "你是一個聰明的 AI 助理，請根據你的知識來回答問題。"},
            {"role": "user", "content": f"{query}"}
        ]
        mode = "🌐 answer with GPT（non-RAG mode）"

    # response with the GPT
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages
    )
    answer = response.choices[0].message.content
    print(mode)
    return answer

In [50]:
if os.path.exists("docs_chunks.txt") and os.path.exists("docs_index.faiss"):
    answer = answer_query("請介紹NovaTech的主要產品")
    print(answer)
else:
    print("⚠️ 尚未成功建立索引或上傳文檔，請先執行對應步驟")

Top similarity distance: 0.7695
Top matched paragraph:
NovaTech 是一家專注於人工智慧、雲端運算與物聯網整合的創新科技公司。我們致力於為企業提供高效、可擴展且智慧化的解決方案，協助客戶在數位轉型的浪潮中取得競爭優勢。

📄 answer with RAG
根據你提供的簡介，NovaTech 的官方產品清單尚未明確列出。基於其專長領域（人工智慧、雲端運算、物聯網整合），以下是可能成為其核心產品方向的推測性分類，供你作為撰寫宣傳素材的參考。這些並非官方確認的產品名稱，而是與公司定位相符的常見解決方案類型：

- AI 與自動化平台
  - 功能方向：整合資料管線、機器學習模型訓練與部署、推理服務、商業流程自動化與工作流管理，具備企業級的模型治理與 MLOps 能力。

- 雲端原生解決方案
  - 功能方向：多雲/混合雲部署、資料湖與資料管線、應用與資料治理、可擴展的雲端架構與 DevOps 能力，支援企業級可擴展性與韌性。

- 物聯網整合與裝置管理
  - 功能方向：裝置連接與註冊、遠端監控、資料收集與轉換、事件驅動自動化、裝置安全與合規性管理。

- 邊緣運算與實時分析
  - 功能方向：在現場或邊緣節點進行資料處理與分析、低延遲決策支援、雲端協同與資料同步。

- 資安與資料治理
  - 功能方向：身份與存取管理、資料加密與保密、風險監控、合規性與審計支援、資料治理框架。

如需，我可以根據你提供的官方資料，幫你整理成正式的「產品概述」或「宣傳文案」，並協助撰寫每個產品方向的重點功能、適用產業與價值主張。需要你提供更具體的官方產品清單或品牌用語後，我就能幫你產出更精準的內容。
